In [1]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 34.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


In [2]:
import mysql.connector
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import time
import warnings
warnings.filterwarnings("ignore")


# Function to update the model with new data
def update_model(model, new_data, scaler):
    # Assuming 'timestamp_column' is the timestamp and 'value_column' is the value to be analyzed
    new_data['post_date'] = pd.to_datetime(new_data['post_date'])
    new_data.set_index('post_date', inplace=True)

    # Check if the scaler is fitted
    # Fit the scaler with training data if not already fitted
    if not hasattr(scaler, 'mean_'):
      training_data = pd.DataFrame({'quantity': [0]})  # Create a dummy DataFrame with the correct column name
      scaler.fit(training_data[['quantity']])


    # Normalize the new data using the fitted scaler
    new_data['quantity'].fillna(0, inplace=True)
    new_data_normalized = new_data.copy()  # Create a copy to keep the original 'post_date' column
    new_data_normalized['quantity'] = scaler.transform(new_data['quantity'].values.reshape(-1, 1))

    # Retrain the model with new data
    model = IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=1.0)  # Create a new model (you may need to adjust hyperparameters)
    model.fit(new_data_normalized[['quantity']])

    return model

# Connect to the MySQL database
conn = mysql.connector.connect(
    host="171.244.188.27",
    database="data_sky007v2",
    user="actsonevn",
    password="actsone123!"
)

# Initialize the scaler and the model
scaler = StandardScaler()
model = IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=1.0)  # Adjust the contamination parameter based on your anomaly rate

# Main loop to continuously fetch and process new data
while True:
    sql_query = "SELECT \
        MAX(CASE WHEN meta_key = '_product_id' THEN meta_value END) AS item_id, \
        MAX(CASE WHEN meta_key = '_qty' THEN meta_value END) AS quantity, \
        p.post_date, oi.order_item_name \
    FROM aowp_posts AS p \
    JOIN aowp_woocommerce_order_items AS oi ON p.ID = oi.order_id \
    JOIN `aowp_woocommerce_order_itemmeta` AS oim ON oi.order_item_id = oim.order_item_id \
    WHERE p.post_date BETWEEN '2015-04-02 00:00:00' AND NOW() \
    GROUP BY p.id"

    # Load new data from the database into a Pandas DataFrame
    new_data = pd.read_sql_query(sql_query, conn)

    # Update the model with new data and get the new model
    model = update_model(model, new_data, scaler)


    # Optionally, perform real-time anomaly detection and take actions based on the results
    # new_data['anomaly'] = model.predict(new_data[['quantity']])
    # anomalies = new_data[new_data['anomaly'] == -1]
    # # Optionally, perform real-time anomaly detection and take actions based on the results
    # new_data['anomaly'] = model.predict(new_data[['quantity']])
    # anomalies = new_data[new_data['anomaly'] == -1]

    #   # Check if 'post_date' and 'quantity' are present in the anomalies DataFrame
    # if 'post_date' in anomalies.columns and 'quantity' in anomalies.columns:
    #     for index, row in anomalies.iterrows():
    #         print(f"Anomaly detected at {row['post_date']} with value {row['quantity']}")
    # else:
    #     print("No 'post_date' or 'quantity' column in anomalies DataFrame.")

    anomaly_predictions = model.predict(new_data[['quantity']])
    new_data['anomaly'] = anomaly_predictions
    anomalies = new_data.loc[anomaly_predictions == -1, ['quantity']]

    # Check if 'post_date' is present in the index names
    if 'post_date' in new_data.index.names:
        for index, row in anomalies.iterrows():
            print(f"Anomaly detected at {index} with value {row['quantity']}")
    else:
        print("No 'post_date' information in anomalies DataFrame.")




# ...

    # new_data['anomaly'] = model.predict(new_data[['quantity']])
    # anomalies = new_data[new_data['anomaly'] == -1]
    # for index, row in anomalies.iterrows():
    #     print(f"Anomaly detected at {row['post_date']} with value {row['quantity']}")

    # Sleep for a specific interval before fetching new data again
    time.sleep(60)  # Sleep for 60 seconds (adjust as needed)


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Anomaly detected at 2022-11-29 14:30:09 with value 2
Anomaly detected at 2022-11-29 15:12:40 with value 2
Anomaly detected at 2022-11-29 15:19:49 with value 2
Anomaly detected at 2022-11-29 16:02:58 with value 3
Anomaly detected at 2022-11-29 16:21:29 with value 2
Anomaly detected at 2022-11-29 17:15:50 with value 2
Anomaly detected at 2022-11-29 18:59:46 with value 2
Anomaly detected at 2022-11-29 19:28:15 with value 2
Anomaly detected at 2022-11-29 19:45:01 with value 2
Anomaly detected at 2022-11-29 20:40:31 with value 2
Anomaly detected at 2022-11-29 21:52:07 with value 2
Anomaly detected at 2022-11-29 22:06:01 with value 5
Anomaly detected at 2022-11-29 22:28:51 with value 2
Anomaly detected at 2022-11-29 22:45:58 with value 3
Anomaly detected at 2022-11-29 23:12:42 with value 2
Anomaly detected at 2022-11-29 23:40:31 with value 2
Anomaly detected at 2022-11-30 00:04:40 with value 2
Anomaly detected at 2022-11-30 00:42:17 wi

KeyboardInterrupt: ignored

In [ ]:
import mysql.connector

# Replace these values with your own database credentials
host = "171.244.188.27"
database = "data_sky007v2"
user = "actsonevn"
password = "actsone123!"

# Establish a connection
conn = mysql.connector.connect(
    host=host,
    database=database,
    user=user,
    password=password
)

# Create a cursor
cursor = conn.cursor()

# Now you can execute SQL queries using the cursor


In [ ]:
# Example query
query = "SELECT \
        MAX(CASE WHEN meta_key = '_product_id' THEN meta_value END) AS item_id, \
        MAX(CASE WHEN meta_key = '_qty' THEN meta_value END) AS quantity, \
        p.post_date, oi.order_item_name \
    FROM aowp_posts AS p \
    JOIN aowp_woocommerce_order_items AS oi ON p.ID = oi.order_id \
    JOIN `aowp_woocommerce_order_itemmeta` AS oim ON oi.order_item_id = oim.order_item_id \
    WHERE p.post_date BETWEEN '2015-04-02 00:00:00' AND NOW() \
    GROUP BY p.id"

# Execute the query
cursor.execute(query)

# Fetch the results
# results = cursor.fetchall()

results = cursor.fetchmany(5)

# Display the results
for row in results:
    print(row)


('101259', '1', datetime.datetime(2019, 1, 1, 9, 46, 21), 'Bbia Last Velvet Lip Tint – #14 Chill Boss')
('34332', '1', datetime.datetime(2019, 1, 1, 10, 30, 39), 'Eglips Lively Liquid Glitter Shadow - #04 Metallic Rose')
('101255', '1', datetime.datetime(2019, 1, 1, 11, 23, 18), 'Bbia Last Velvet Lip Tint – #11 Calm Boss')
('136270', '1', datetime.datetime(2019, 1, 1, 11, 45, 44), 'Bbia Last Velvet Lip Tint - #25 Final Note')
('170926', '1', datetime.datetime(2019, 1, 1, 12, 14, 45), 'Eglips Calendar 2019')
